In [233]:
import pandas as pd
import numpy as np

In [251]:
df_tour = pd.read_csv('./busan_tourlist.csv', encoding='UTF-8', index_col=0)

In [252]:
df_tour.head(2)

,name,GUN,address,lat,lng,tel,hompage,holiday,image,explain
0,흰여울문화마을,영도구,부산광역시 영도구 흰여울길,35.078850,129.04402,051-419-4067,http://www.ydculture.com/huinnyeoulculturetown/,NaN,https://www.visitbusan.net/uploadImgs/files/cn...,절영해안산책로 가파른 담벼락 위로 독특한 마을 풍경이 보인다. 해안가 절벽 끝에 바...
1,깡깡이 예술마을,영도구,부산시 영도구 대평북로 36 깡깡이 안내센터,35.092648,129.03255,051-418-3336,http://kangkangee.com,매주 월요일 (투어 등 시설),https://www.visitbusan.net/uploadImgs/files/cn...,영도다리 건너 자갈치시장 맞은 편 물양장에 배들이 가득 들어차 있다. 세상에 못 고...


In [253]:
# 필요없는 부분 삭제
df_tour.drop(['GUN'], axis=1, inplace=True)
df_tour.drop(['hompage'], axis=1, inplace=True)

In [254]:
# 필요한 열 생성
df_tour['category'] = ""
df_tour['label'] = '관광지'

In [255]:
# 열 이름 변경
df_tour.rename(columns={'tel': 'phone'}, inplace=True)
df_tour.rename(columns={'holiday': 'opentime'}, inplace=True)
df_tour.rename(columns={'image': 'thumbnail'}, inplace=True)
df_tour.rename(columns={'explain': 'info'}, inplace=True)

In [240]:
# 열 재배치
df_tour = df_tour[['name', 'info', 'opentime', 'lat', 'lng', 'address', 'category', 'phone', 'label', 'station', 'average_score', 'thumbnail']]

- 리뷰 데이터 합치기

In [243]:
# 관광지목록
df_tour = pd.read_csv('./busan_tour_change.csv', encoding='cp949', index_col=0)
# 카카오 관광지 리뷰
df_review = pd.read_csv('./tour_review.csv', encoding='UTF-8', index_col=0)

In [145]:
# 네이버 리뷰
review_list = pd.read_csv('./부산리뷰.csv', encoding='UTF-8', index_col=0)

In [151]:
review_list.rename(columns={'score': 'rank'}, inplace=True)
review_list.rename(columns={'name': 'title'}, inplace=True)

In [162]:
df_review.rename(columns={'name': 'title'}, inplace=True)
df_review.rename(columns={'score': 'rank'}, inplace=True)

In [165]:
# 데이터 합치기
result = pd.concat([df_review, review_list], ignore_index=True)

In [210]:
# 데이터 프레임 생성
columns = ['title', 'subway', 'rank_average', 'rank_count']
review_df = pd.DataFrame(columns=columns)

In [205]:
tour_list = list(df_tour['name'].value_counts().index)
tour_list2 = list(result['title'].value_counts().index)

- 별점 계산해주기

In [211]:
for tour in tour_list:
    tour_subway = ''
    if len(result[result['title'] == tour]) == 0:
        continue
    new_tour = result[result['title'] == tour]
    tour_name = new_tour['title'].iloc[0]
    
#     print(tour_subway)
    null_list = new_tour[new_tour['subway'].isnull() == False]
    if len(null_list) == 0:
        pass
    else:
        tour_subway = null_list['subway'].iloc[0]
    
    rank_list = new_tour[np.isnan(new_tour['rank']) == False]
#     print(rank_list)
#     c = rank_list[np.isnan(rank_list['rank']) == True]
    rank_count = len(rank_list)
    rank_sum = rank_list['rank'].sum()
#     print(rank_sum)
#     print(rank_count)
    rank_average = round(rank_sum / rank_count, 2)
#     print(rank_average)
    
    review_df = review_df.append({
        'title': tour_name,
        'subway': tour_subway,
        'rank_average': rank_average,
        'rank_count': rank_count,
    },ignore_index=True
    )

In [229]:
# 열 이름 조정
review_df.rename(columns={'title': 'name'}, inplace=True)
review_df.rename(columns={'rank_average': 'average_score'}, inplace=True)
review_df.rename(columns={'subway': 'station'}, inplace=True)

In [256]:
# 정보데이터와 리뷰 개수 데이터 합치기
info_result = pd.merge(df_tour, review_df, how="outer", on='name')

In [263]:
info_result.rename(columns={'rank_count': 'review_count'}, inplace=True)
info_result = info_result[['name', 'info', 'opentime', 'lat', 'lng', 'address', 'category', 'phone', 'label', 'station', 'average_score', 'thumbnail', 'review_count']]

In [269]:
info_result.to_csv(path_or_buf='busan_tour_change2.csv', encoding='utf-8-sig')